In [15]:
from PIL import Image
from torchvision import transforms
import timm
import torch
import torch.nn as nn

# Load model from Hugging Face Hub
model = timm.create_model("hf_hub:Snarcy/RedDino-base", pretrained=True)
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load and preprocess image
image = Image.open("C:/Code/DL/bbosis/Hongyuan-Babesiosis/data/core_data/2.jpg").convert("RGB")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])
input_tensor = transform(image).unsqueeze(0).to(device)

# Extract features
with torch.no_grad():
    embedding = model(input_tensor)


In [13]:
embedding
embedding.size()

torch.Size([1, 768])

In [33]:
class ClassifierHead(nn.Module):
    def __init__(self, base_model, num_classes):
        super().__init__()
        self.base_model = base_model
        # The RedDino model outputs [batch_size, num_tokens, feature_dim]
        # where num_tokens = 257 (1 CLS + 256 patches) and feature_dim = 768
        self.l1 = nn.Linear(768, 512)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(512, 512)
        self.relu = nn.ReLU()
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        # Get features from base model
        features = self.base_model.forward_features(x)  # [batch_size, 257, 768]
        
        # Use CLS token (first token) for classification
        cls_token = features[:, 0, :]  # [batch_size, 768]
        cls_token = self.l1(cls_token)
        cls_token = self.relu(cls_token)

        
        # Apply classifier
        logits = self.classifier(cls_token)
        return logits
    
num_classes = 2
m_model = ClassifierHead(model, num_classes).to(device)
print(f"Model created with {num_classes} classes")

# Test the model with a sample batch
with torch.no_grad():
    sample_batch = next(iter(train_loader))
    sample_images, sample_labels = sample_batch
    sample_images = sample_images.to(device)
    outputs = m_model(sample_images)
    print(f"Model output shape: {outputs.shape}")
    print(f"Sample labels shape: {sample_labels.shape}")
    print(f"Output sample: {outputs[:3]}")  # First 3 predictions

Model created with 2 classes
Model output shape: torch.Size([10, 2])
Sample labels shape: torch.Size([10])
Model output shape: torch.Size([10, 2])
Sample labels shape: torch.Size([10])
Output sample: tensor([[0.0196, 0.0028],
        [0.0202, 0.0036],
        [0.0200, 0.0034]], device='cuda:0')
Output sample: tensor([[0.0196, 0.0028],
        [0.0202, 0.0036],
        [0.0200, 0.0034]], device='cuda:0')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
from pathlib import Path
import os

def train_model(
    model,
    train_loader,
    test_loader,
    num_epochs=2,
    learning_rate=0.001,
    checkpoint_dir='checkpoints',
    device='cuda'
):
    """
    Training loop with GPU support, logging, and checkpointing.
    
    Args:
        model: Your ClassifierHead model
        train_loader: DataLoader for training data
        test_loader: DataLoader for test data
        num_epochs: Number of training epochs
        learning_rate: Learning rate for optimizer
        checkpoint_dir: Directory to save model checkpoints
        device: 'cuda' or 'cpu'
    """
    # Setup
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Create checkpoint directory
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Training loop
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # Move data to device
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Statistics
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
            
            # Print progress every 10 batches
            if batch_idx % 10 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}, Loss: {loss.item():.4f}')
        
        # Calculate training metrics
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100. * train_correct / train_total
        
        # Evaluation phase
        model.eval()
        test_correct = 0
        test_total = 0
        test_loss = 0.0
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                test_total += labels.size(0)
                test_correct += predicted.eq(labels).sum().item()
        
        # Calculate test metrics
        avg_test_loss = test_loss / len(test_loader)
        test_accuracy = 100. * test_correct / test_total
        
        # Logging
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'  Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.2f}%')
        print(f'  Test Loss: {avg_test_loss:.4f} | Test Acc: {test_accuracy:.2f}%')
        
        # Save checkpoint every 5 epochs
        if (epoch + 1) % 5 == 0:
            checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch+1}.pt')
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': avg_train_loss,
                'test_loss': avg_test_loss,
                'test_accuracy': test_accuracy,
            }, checkpoint_path)
            print(f'  Checkpoint saved: {checkpoint_path}')
        
        print('-' * 60)
    
    # Save final model
    final_path = os.path.join(checkpoint_dir, 'model_final.pt')
    torch.save({
        'epoch': num_epochs,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': avg_train_loss,
        'test_loss': avg_test_loss,
        'test_accuracy': test_accuracy,
    }, final_path)
    print(f'Final model saved: {final_path}')
    

In [35]:
from torch.utils.data import Dataset
from pathlib import Path

class ImageDataset(Dataset):
    def __init__(self, csv_path, transform=None):
        self.data = pd.read_csv(csv_path)
        self.transform = transform
        
        # Create label mapping
        unique_labels = self.data['label'].unique()
        self.label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
        print(f"Label mapping: {self.label_to_idx}")
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['img_path']
        label_str = self.data.iloc[idx]['label']
        
        # Convert string label to numeric
        label = self.label_to_idx[label_str]
        
        # Load image with correct path
        full_path = Path(r'C:\Code\DL\bbosis\Hongyuan-Babesiosis\data') / img_path
        image = Image.open(full_path).convert("RGB")
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label, dtype=torch.long)

# Create dataset with the transform
csv_path = r'C:\Code\DL\bbosis\Hongyuan-Babesiosis\data\training.csv'
dataset = ImageDataset(csv_path, transform=transform)

# Split into train/test (80/20)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Train the model
train_model(
    model=m_model,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=15,
    learning_rate=0.001,
    checkpoint_dir='checkpoints',
    device=device
)

Label mapping: {'NTTGS': 0, 'BABSP': 1}
Using device: cuda
torch.Size([4, 3, 224, 224]) <class 'torch.Tensor'>
torch.Size([4, 3, 224, 224]) <class 'torch.Tensor'>
torch.Size([2, 3, 224, 224]) <class 'torch.Tensor'>
torch.Size([4, 3, 224, 224]) <class 'torch.Tensor'>
torch.Size([2, 3, 224, 224]) <class 'torch.Tensor'>
Epoch [1/15]
  Train Loss: 0.8235 | Train Acc: 60.00%
  Test Loss: 0.7515 | Test Acc: 0.00%
------------------------------------------------------------
Epoch [1/15]
  Train Loss: 0.8235 | Train Acc: 60.00%
  Test Loss: 0.7515 | Test Acc: 0.00%
------------------------------------------------------------
torch.Size([4, 3, 224, 224]) <class 'torch.Tensor'>
torch.Size([4, 3, 224, 224]) <class 'torch.Tensor'>
torch.Size([4, 3, 224, 224]) <class 'torch.Tensor'>
torch.Size([2, 3, 224, 224]) <class 'torch.Tensor'>
torch.Size([4, 3, 224, 224]) <class 'torch.Tensor'>
torch.Size([2, 3, 224, 224]) <class 'torch.Tensor'>
Epoch [2/15]
  Train Loss: 0.7064 | Train Acc: 40.00%
  Test Lo

In [27]:
# Debug the model output shapes
with torch.no_grad():
    test_input = torch.randn(1, 3, 224, 224).to(device)
    print(f"Input shape: {test_input.shape}")
    
    # Test base model features
    features = model.forward_features(test_input)
    print(f"Features shape: {features.shape}")
    print(f"Features type: {type(features)}")
    
    # Test a small batch from dataset
    sample_batch = next(iter(train_loader))
    sample_images, sample_labels = sample_batch
    print(f"Sample batch shapes - Images: {sample_images.shape}, Labels: {sample_labels.shape}")
    
    sample_images = sample_images.to(device)
    sample_features = model.forward_features(sample_images)
    print(f"Sample features shape: {sample_features.shape}")

Input shape: torch.Size([1, 3, 224, 224])
Features shape: torch.Size([1, 257, 768])
Features type: <class 'torch.Tensor'>
Sample batch shapes - Images: torch.Size([10, 3, 224, 224]), Labels: torch.Size([10])
Sample features shape: torch.Size([10, 257, 768])
Sample batch shapes - Images: torch.Size([10, 3, 224, 224]), Labels: torch.Size([10])
Sample features shape: torch.Size([10, 257, 768])
